In [7]:
from operator import itemgetter
from utils import merge_utils as mu
from utils import write_utils as wu

In [2]:
def interleave(sc,submissions_list):
    
    merge_rdd = sc.emptyRDD()
    
    for s in submissions_list:
        ls_rdd = parse_submission_with_value(sc,s[0],s[1])
        merge_rdd = merge_rdd.union(ls_rdd)
    
    
    merge_rdd = (merge_rdd.flatMap(lambda r: [(r[0], x) for x in r[1]])
                 .map(lambda x: ((x[0],x[1][0]),x[1][1]))
                 .reduceByKey(lambda x,y:max(x,y))
                 .map(lambda x: (x[0][0],[(x[0][1],x[1])]))
                 .reduceByKey(lambda x,y:x+y)
                 .map(lambda x: (x[0],sorted(x[1], key=itemgetter(1), reverse=True)[:30]))
                 )
    
    return merge_rdd

def recommendation_list_with_value(string, value):
    updating_value = value
    lista=[]
    split = string.split(',')
    for i in split:
        if i != '':
            lista.append((int(i),updating_value))
            updating_value -= 1
            
    return lista

def parse_submission_with_value(sc, path, value):
    rdd = sc.textFile(path)
    
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
    .map(lambda x: x.split("\t"))
    .filter(lambda x: x[0] != '')
    .map(lambda x:(int(x[0]),recommendation_list_with_value(x[1], value)))
    )

    return rdd

In [3]:
def recommendation_list(string):
    lista=[]
    split = string.split(',')
    for i in split:
        if i != '':
            lista.append((int(i)))
            
    return lista

def parse_submission(sc, path):
    rdd = sc.textFile(path)
    
    header = rdd.first()

    rdd = (rdd.filter(lambda x: x != header)
    .map(lambda x: x.split("\t"))
    .filter(lambda x: x[0] != '')
    .map(lambda x:(int(x[0]),recommendation_list(x[1])))
    )

    return rdd

In [5]:
recommendation = interleave(sc, [("../../Submissions/ens_subs/interactions_82k.csv",30),
                                 ("../../Submissions/ens_subs/impressions_116k.csv",30),
                                 ("../../Submissions/ens_subs/cf_int_int_18.9k.csv",30),
                                 ("../../Submissions/ens_subs/cf_int_imp_22.4k.csv",30),
                                 ("../../Submissions/ens_subs/imp_imp_int_int_33.2k_avg.csv",30),
                                ("../../Submissions/ens_subs/cf_imp_imp_31k.csv",30),
                                ("../../Submissions/ens_subs/true_10k_CF_ITEM_greater_3_shr=1_knn=750_22k.csv",30),
                                ("../../Submissions/ens_subs/cf_item_imp_imp_20k.csv",30),
                                ("../../Submissions/ens_subs/idf_45k.csv",30),
                                ("../../Submissions/ens_subs/tf_ervin_37k.csv",30)])
print recommendation.take(1)

[(76800, [(2395026, 30), (1793593, 30), (2293641, 30), (784737, 30), (1063437, 30), (2224944, 30), (2477853, 30), (398809, 30), (2337484, 30), (1617570, 29), (1848945, 29), (1208167, 29), (956825, 29), (1244196, 29), (490302, 29), (862730, 29), (326608, 29), (880348, 28), (282769, 28), (2755404, 28), (671708, 28), (536047, 28), (976877, 28), (1994213, 28), (1442340, 28), (155380, 28), (474030, 27), (964497, 27), (1577023, 27), (1477419, 27)])]


In [8]:
wu.prepareSubmission(recommendation,"results/pure_interleaving")

Submission file results/pure_interleaving created successfully
